In [8]:
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import (train_test_split, GridSearchCV)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (StandardScaler, OneHotEncoder, FunctionTransformer, KBinsDiscretizer)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy import stats
import numpy as np
import pandas as pd

# Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# plot correlations
def plotCoorelations(df):
    # remove non_numeric features  
    corr = df.corr()
    corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [3]:
# define a method to use Isolation Forest for outlier detection
def outlierDetect_IsolationForest(home_data):
    X_numeric = home_data.select_dtypes(exclude=object) 
    imp_mean = SimpleImputer()
    X_numeric_t = imp_mean.fit_transform(X_numeric)
    clf = IsolationForest( behaviour = 'new', contamination= 0.02)
    preds = clf.fit_predict(X_numeric_t)
    outliers = np.where(preds == -1)

    # drop outliers from home data
    home_data_out = home_data.drop(labels = outliers[0], inplace=False, errors = "ignore")
    #home_data_out = home_data[(preds == -1).all(axis=1)]
    print("number of outliers = {0}".format(len(outliers[0])))
    return home_data_out, outliers
    

In [4]:
# define a method to use Isolation Forest for outlier detection
def outlierDetect_ZScore(home_data, zValue = 3):
    X_numeric = home_data.select_dtypes(exclude=object) 
    
    z = np.abs(stats.zscore(X_numeric))
    outliers = np.where(z > zValue)

    # drop outliers from home data
    home_data_out = home_data.drop(labels = outliers[0], inplace=False, errors = "ignore")
    #home_data_out = home_data[(preds == -1).all(axis=1)]
    print("number of outliers = {0}".format(len(outliers[0])))
    return home_data_out, outliers

In [5]:
# load housing data
iowa_file_path = '../data/train.csv'
home_data = pd.read_csv(iowa_file_path)
Y = home_data["SalePrice"]
X = home_data.drop(columns = ["Id"])

# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=1)
print(X_train.shape)

# look at coorelations
#plotCoorelations(X_train)

# perform mean normalization and feature scaling on the data

home_data, dummy = outlierDetect_IsolationForest(home_data.copy())
#home_data, dummy = outlierDetect_ZScore(X_train.copy(), 6)


(1314, 80)
number of outliers = 30


In [16]:
# We will train our classifier with the following features:

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = X_train.select_dtypes(exclude=object) 
num_features_names = numeric_features.columns

# features that need a log transformation
log_features_names = ["LotFrontage", "LotArea", "1stFlrSF", "GrLivArea", "OpenPorchSF"]

log_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', FunctionTransformer(np.log1p, validate=False)),
    ('logscaler', StandardScaler())])

#kbinDiscretizer features
year_features_names = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']
year_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('kbd', KBinsDiscretizer(n_bins=5, encode='onehot-dense'))])

#numeric features that require a normal transformation

numeric_features_names = [x for x in num_features_names if x not in log_features_names + year_features_names]
print(len(numeric_features_names))

numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features_names = X_train.select_dtypes(include=object).columns

cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformers=[
        ('log', log_pipeline, log_features_names),
        ('num', numeric_pipeline, numeric_features_names),
        ('cat', cat_pipeline, categorical_features_names),
        ('year', year_pipeline, year_features_names)     
    ]

ct = ColumnTransformer(transformers=transformers)
Xt = ct.fit_transform(X_train)
print(Xt.shape)

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', ct),
                      ('classifier', RandomForestRegressor(random_state=1, n_estimators = 500, criterion="mae"))
                     ])



28
(1314, 319)


c:\users\rajesh\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\preprocessing\_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 3 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


In [17]:
#fitting and prediction
clf.fit_transform(X_train, y_train)
y_predict = clf.predict(X_test)
rf_val_mae = mean_absolute_error(y_test, y_predict)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))

#train the best model on the full dat
clf.fit(X,Y)

#path to file you will use for predictions
test_data_path = '../data/test.csv'
test_data = pd.read_csv(test_data_path)

X_test2 = test_data.drop(columns = ["Id"])

#make predictions which we will submit. 
y_pred2 = clf.predict(X_test2)

#The lines below shows how to save predictions in format used for competition scoring
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': y_pred2})
output.to_csv('../data/submission.csv', index=False)

c:\users\rajesh\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\preprocessing\_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 3 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


KeyboardInterrupt: 